In [185]:
import pandas as pd
import os
pd.set_option('display.max_rows', 200)

In [127]:
data_dir = "/Users/mengzhouxia/dongdong/Princeton/Courses/COS597E/Project/597E-assignment1/data/state-level"

In [132]:
files = {"uva": "uva.csv",
        "economist": "economist.csv",
        "538": "fivethirtyeight.csv",
        "pec": "pec.csv",
        "pollyvote": "pollyvote.csv"}

In [267]:
data = {file: pd.read_csv(os.path.join(data_dir, files[file])) for file in files}

In [134]:
for file in data:
    print(file, data[file].columns.values)

uva ['Date' 'office' 'office.1' 'state' 'party' 'candidate' 'prediction']
economist ['date' 'model' 'office' 'state' 'party' 'candidate' 'win_prob' 'est_diff'
 'est_share' 'est_share_2p']
538 ['date' 'model' 'office' 'state' 'party' 'candidate' 'win_prob' 'est_diff'
 'est_share' 'est_share_2p']
pec ['date' 'model' 'office' 'state' 'party' 'candidate' 'win_prob' 'est_diff'
 'est_share' 'est_share_2p']
pollyvote ['date' 'model' 'office' 'state' 'party' 'candidate' 'win_prob' 'est_diff'
 'est_share' 'est_share_2p']


In [151]:
data["pollyvote"].state.unique()

array(['MN', 'NH', 'ME', 'NV', 'CA', 'CO', 'CT', 'DC', 'DE', 'HI', 'IL',
       'MA', 'MD', 'NJ', 'NM', 'NY', 'OR', 'RI', 'VA', 'VT', 'WA', 'MI',
       'PA', 'AZ', 'FL', 'NC', 'WI', 'GA', 'IA', 'OH', 'TX', 'AK', 'AL',
       'AR', 'ID', 'IN', 'KS', 'KY', 'LA', 'MO', 'MS', 'MT', 'ND', 'NE',
       'OK', 'SC', 'SD', 'TN', 'UT', 'WV', 'WY', 'NE_01', 'NE_02',
       'NE_03', 'ME_01', 'ME_02'], dtype=object)

In [29]:
def get_latest_forecasts(df):
    return df.sort_values("date")\
        .groupby(["office", "state", "party", "candidate"])\
        .last()\
        .reset_index()


In [206]:
def get_certain(df, model):
    return df[df["model"] == model]

In [125]:
WINNERS_2016_PATH = os.path.join("/Users/mengzhouxia/dongdong/Princeton/Courses/COS597E/Project/2016-11-grading-the-election-forecasts/data/reference", "winners.csv")
WINNERS_2020_PATH = os.path.join("/Users/mengzhouxia/dongdong/Princeton/Courses/COS597E/Project/597E-assignment1/data/winners.csv")

In [35]:
data_2016 = pd.read_csv("/Users/mengzhouxia/dongdong/Princeton/Courses/COS597E/Project/2016-11-grading-the-election-forecasts/data/forecasts/combined.csv")
latest_2016 = data_2016.sort_values("date")\
        .groupby(["model", "office", "state", "party", "candidate"])\
        .last()\
        .reset_index()

In [44]:
latest_2016_538_poll = get_2016_certain(latest_2016, "538_polls")

In [105]:
TRANSLATIONS = {
    "538_polls": '538 "Polls-only"',
    "538_plus": '538 "Polls-plus"',
    "dailykos": 'Daily Kos',
    "huffpost": 'Huffington Post',
    "nyt": 'NYT/Upshot',
    "pec": 'Princeton E.C.',
    "predictwise_markets": 'PredictWise Markets',
    "predictwise_overall": 'PredictWise Overall',
    "desart": "DeSart and Holbrook",
    "kremp": "Kremp/Slate",
    "pollsavvy": "PollSavvy"
}

latest_2016_538_poll

,model,office,state,party,candidate,date,win_prob,est_diff,est_share,est_share_2p
219,538_polls,P,AK,D,CLINTON,2016-11-07,0.25150,-0.06967,0.41198,0.46102
220,538_polls,P,AK,L,JOHNSON,2016-11-07,0.00055,NaN,0.09045,NaN
221,538_polls,P,AK,R,TRUMP,2016-11-07,0.74795,0.06967,0.48166,0.53898
222,538_polls,P,AL,D,CLINTON,2016-11-07,0.00130,-0.22283,0.35979,0.38178
223,538_polls,P,AL,L,JOHNSON,2016-11-07,0.00000,NaN,0.04011,NaN
...,...,...,...,...,...,...,...,...,...,...
433,538_polls,S,VT,R,MILNE,2016-11-07,0.00215,-0.39626,0.28505,0.29497
434,538_polls,S,WA,D,MURRAY,2016-11-07,0.98365,0.15364,0.57682,0.57682
435,538_polls,S,WA,R,VANCE,2016-11-07,0.01635,-0.15364,0.42318,0.42318
436,538_polls,S,WI,D,FEINGOLD,2016-11-07,0.75860,0.03200,0.50443,0.51638


In [318]:
latest_data = {file: get_latest_forecasts(data[file]) for file in ["538", "economist", "pollyvote", "pec"]}
latest_data = {file: latest_data[file].drop(latest_data[file][~latest_data[file]['state'].isin(ELECTORAL_VOTES.to_dict().keys())].index)  for file in latest_data}

# calculation

In [167]:
ELECTORAL_VOTES = pd.Series({
    "AL": 9, "AK": 3, "AZ": 11, "AR": 6, "CA": 55, "CO": 9, "CT": 7,
    "DE": 3, "DC": 3, "FL": 29, "GA": 16, "HI": 4, "ID": 4, "IL": 20, "IN": 11,
    "IA": 6, "KS": 6, "KY": 8, "LA": 8, "ME": 2, "MD": 10, "MA": 11, "MI": 16,
    "MN": 10, "MS": 6, "MO": 10, "MT": 3, "NE": 2, "NV": 6, "NH": 4, "NJ": 14,
    "NM": 5, "NY": 29, "NC": 15, "ND": 3, "OH": 18, "OK": 7, "OR": 7, "PA": 20,
    "RI": 4, "SC": 9, "SD": 3, "TN": 11, "TX": 38, "UT": 6, "VT": 3,
    "VA": 13, "WA": 12, "WV": 5, "WI": 10, "WY": 3, 
    # "NE-1": 1, "NE-2": 1, "NE-3": 1,
    # "ME-1": 1, "ME-2": 1
})

ELECTORAL_VOTES_pec = pd.Series({
    "AL": 9, "AK": 3, "AZ": 11, "AR": 6, "CA": 55, "CO": 9, "CT": 7,
    "DE": 3, "DC": 3, "FL": 29, "GA": 16, "HI": 4, "ID": 4, "IL": 20, "IN": 11,
    "IA": 6, "KS": 6, "KY": 8, "LA": 8, "ME": 2, "MD": 10, "MA": 11, "MI": 16,
    "MN": 10, "MS": 6, "MO": 10, "MT": 3, "NE": 2, "NV": 6, "NH": 4, "NJ": 14,
    "NM": 5, "NY": 29, "NC": 15, "ND": 3, "OH": 18, "OK": 7, "OR": 7, "PA": 20,
    "RI": 4, "SC": 9, "SD": 3, "TN": 11, "TX": 38, "UT": 6, "VT": 3,
    "VA": 13, "WA": 12, "WV": 5, "WI": 10, "WY": 3, "NE1": 1, "NE2": 1, "NE3": 1,
    "ME1": 1, "ME2": 1
})

ELECTORAL_VOTES_economist = pd.Series({
    "AL": 9, "AK": 3, "AZ": 11, "AR": 6, "CA": 55, "CO": 9, "CT": 7,
    "DE": 3, "DC": 3, "FL": 29, "GA": 16, "HI": 4, "ID": 4, "IL": 20, "IN": 11,
    "IA": 6, "KS": 6, "KY": 8, "LA": 8, "ME": 4, "MD": 10, "MA": 11, "MI": 16,
    "MN": 10, "MS": 6, "MO": 10, "MT": 3, "NE": 5, "NV": 6, "NH": 4, "NJ": 14,
    "NM": 5, "NY": 29, "NC": 15, "ND": 3, "OH": 18, "OK": 7, "OR": 7, "PA": 20,
    "RI": 4, "SC": 9, "SD": 3, "TN": 11, "TX": 38, "UT": 6, "VT": 3,
    "VA": 13, "WA": 12, "WV": 5, "WI": 10, "WY": 3
})

ELECTORAL_VOTES_pollyvote = pd.Series({
    "AL": 9, "AK": 3, "AZ": 11, "AR": 6, "CA": 55, "CO": 9, "CT": 7,
    "DE": 3, "DC": 3, "FL": 29, "GA": 16, "HI": 4, "ID": 4, "IL": 20, "IN": 11,
    "IA": 6, "KS": 6, "KY": 8, "LA": 8, "ME": 4, "MD": 10, "MA": 11, "MI": 16,
    "MN": 10, "MS": 6, "MO": 10, "MT": 3, "NE": 5, "NV": 6, "NH": 4, "NJ": 14,
    "NM": 5, "NY": 29, "NC": 15, "ND": 3, "OH": 18, "OK": 7, "OR": 7, "PA": 20,
    "RI": 4, "SC": 9, "SD": 3, "TN": 11, "TX": 38, "UT": 6, "VT": 3,
    "VA": 13, "WA": 12, "WV": 5, "WI": 10, "WY": 3, "NE01": 1, "NE02": 1, "NE03": 1,
    "ME01": 1, "ME02": 1
})

'polls-plus', 'economist', 'pollyvote', 'pec'
def get_evs(abbr, model=None):
    if model == "polls-plus":
        return ELECTORAL_VOTES_538.get(abbr)
    elif model == 'economist':
        return ELECTORAL_VOTES_economist.get(abbr)
    elif model == 'pollyvote':
        return ELECTORAL_VOTES_pollyvote.get(abbr)
    elif model == 'pec':
        return ELECTORAL_VOTES_pec.get(abbr)
    else:
        return ELECTORAL_VOTES.get(abbr)
#     return ELECTORAL_VOTES.get(abbr)

In [47]:
def brier_score_race(called_forecast):
    uniques = called_forecast[[
        "date", "office", "state", "model"
    ]].apply(lambda x: x.nunique())
    assert((uniques != 1).sum() == 0)
    errors = called_forecast["win_prob"] - called_forecast["correct"]
    errors_squared = (errors).pow(2).sum()
    # If you didn't put odds on the candidate (e.g., McMullin in UT, add 1)
    if called_forecast["correct"].sum() == 0:
        errors_squared += 1
    return errors_squared / 2

In [253]:
class Scenario(object):
    def __init__(self, winners, forecasts):
        self.winners = winners
        self.forecasts = forecasts
        self.called = self.judge_called_races()
        self.credits = self.calc_credits()
        self.brier_scores = self.calc_brier_scores()
        self.log_scores = self.calc_log_scores()
        self.forecast_scores = self.calc_forecast_scores()
    
    @classmethod
    def from_csv(cls, path, forecasts):
        winners = pd.read_csv(path)
        return cls(winners, forecasts)
    
    @property
    def uncalled_races(self):
        called_races = self.winners[
            self.winners["winner"].notnull()
        ].set_index(["office", "state"]).index
        # Assume Gary Johnson has zero chance of winning any state
        unique_races = latest_forecasts[
            ~((latest_forecasts["candidate"] == "JOHNSON") &
            (latest_forecasts["office"] == "P"))
        ].groupby(["office", "state"])["candidate"].unique()
        return unique_races[
            ~unique_races.index.isin(called_races)
        ]
    
    @property
    def evs_called(self):
        return pd.Series(self.winners[
            (self.winners["office"] == "P") &
            (self.winners["winner"].notnull())
        ]["state"].unique()).apply(get_evs).sum()
    
    @property
    def pres_diffs_final(self):
        return len(self.winners[
            (self.winners["office"] == "P") &
            (self.winners["state"] != "DC") &
            (self.winners["dem_diff"].notnull())
        ])
    
    def judge_called_races(self):
        called = pd.merge(
            self.winners,
            self.forecasts,
            on=[ "office", "state" ],
            how="left"
        ).dropna(subset=["winner"])
        called["correct"] = called["winner"].str.upper() == called["candidate"].str.upper()
        return called
    
    def calc_credits(self):
        def get_credits(race):
            max_prob = race["win_prob"].max()
            at_max = race["win_prob"] == max_prob
            favorites = race[at_max]
            credit = (1 / len(favorites)) * favorites["correct"]
            return credit.sum()
        return pd.DataFrame({
            "credit": self.called.groupby([
                    "date", "model", "office", "state"
                ]).apply(get_credits)
        }).reset_index()

    
    def calc_brier_scores(self):
        scores = pd.merge(
            self.called,
            self.forecasts[["date", "model"]].drop_duplicates(),
            how="inner",
            on=["date", "model"]
        ).groupby([
            "date",
            "office",
            "state",
            "model",
        ]).apply(brier_score_race)\
            .reset_index()\
            .rename(columns={0: "brier_score"})
        scores.loc[(
            (scores["office"] == "P")
        ), "brier_evs"] = scores.apply(lambda x: x["brier_score"] * get_evs(x["state"]), axis=1)
        return scores

    def calc_log_scores(self):
        # NOTE: To account for the various levels of precision reported,
        # these calculations convert 0% odds to 0.01% odds (i.e., 1 in 10,000).
        scores = pd.merge(
            self.called[self.called["candidate"] == self.called["winner"]],
            self.forecasts[["date", "model", ]].drop_duplicates(),
            how="inner",
            on=["date", "model"]
        ).groupby([
            "date",
            "office",
            "state",
            "model",
            "win_prob"
        ]).apply(lambda x: pd.np.math.log(max(x["win_prob"].iloc[0], 0.0001)))\
            .reset_index()\
            .rename(columns={0: "log_score"})
        scores.loc[(
            (scores["office"] == "P")
        ), "log_evs"] = scores.apply(lambda x: x["log_score"] * get_evs(x["state"]), axis=1)
        return scores

    def calc_squared_errors_share(self):
        errors = pd.merge(
            self.called[
                (self.called["office"] == "P") &
                (self.called["party"] == "D") &
                ~(self.called["state"].isin([ "DC", "UT" ]))
            ],
            self.forecasts[["date", "model"]].drop_duplicates(),
            how="inner",
            on=["date", "model"]
        ).groupby([
            "date",
            "office",
            "state",
            "model",
            "est_share_2p"
        ]).apply(lambda x: pow(x.iloc[0]["est_share_2p"] - x.iloc[0]["dem_share_2p"], 2))\
            .reset_index()\
            .rename(columns={0: "squared_error"})
        return errors
    
    def calc_squared_errors_diff(self):
        errors = pd.merge(
            self.called[
                (self.called["office"] == "P") &
                (self.called["party"] == "D") &
                ~(self.called["state"].isin([ "DC", "UT" ]))
            ],
            self.forecasts[["date", "model"]].drop_duplicates(),
            how="inner",
            on=["date", "model"]
        ).groupby([
            "date",
            "office",
            "state",
            "model",
            "est_diff"
        ]).apply(lambda x: pow(x.iloc[0]["est_diff"] - x.iloc[0]["dem_diff"], 2))\
            .reset_index()\
            .rename(columns={0: "squared_error"})
        return errors

    def biggest_misses(self, val):
        return {
            "brier_score": self.brier_scores,
            "brier_evs": self.brier_scores,
            "log_score": self.log_scores,
            "log_evs": self.log_scores,
        }[val].sort_values(val, ascending=False if "brier" in val else True)\
            .reset_index(drop=True)[[ "date", "model", "office", "state", val]]
    
    def calc_forecast_scores(self):
        pb = self.brier_scores[self.brier_scores["office"] == "P"]
        sb = self.brier_scores[self.brier_scores["office"] == "S"]
        pl = self.log_scores[self.log_scores["office"] == "P"]
        sl = self.log_scores[self.log_scores["office"] == "S"]
        
        scores = pd.DataFrame({
            "pres_brier_flat": pb.groupby("model")["brier_score"].mean(),
            "pres_brier_evs": pb.groupby("model")["brier_evs"].sum() / self.evs_called,
            "sen_brier_flat": sb.groupby("model")["brier_score"].mean(),
            "pres_log_flat": pl.groupby("model")["log_score"].mean(),
            "pres_log_evs": pl.groupby("model")["log_evs"].sum() / self.evs_called,
            "sen_log_flat": sl.groupby("model")["log_score"].mean(),
            "pres_correct": self.credits[self.credits["office"] == "P"].groupby("model")["credit"].sum(),
            "sen_correct": self.credits[self.credits["office"] == "S"].groupby("model")["credit"].sum()
        })
        return scores

In [162]:
scenario_2016.forecast_scores.round(4)\
    .sort_values("pres_brier_evs").fillna("")\
    [[ "pres_brier_evs", "pres_brier_flat",
       "pres_log_evs", "pres_log_flat", "pres_correct" ]].rename(index=TRANSLATIONS)#.to_csv(sys.stdout)

,pres_brier_evs,pres_brier_flat,pres_log_evs,pres_log_flat,pres_correct
"538 ""Polls-plus""",0.0928,0.0664,-0.2724,-0.2123,46.0
"538 ""Polls-only""",0.0936,0.0672,-0.2749,-0.2160,46.0
Princeton E.C.,0.1169,0.0744,-0.3602,-0.2471,47.0
NYT/Upshot,0.1208,0.0801,-0.3601,-0.2554,46.0
Kremp/Slate,0.1210,0.0766,-0.3873,-0.2595,46.0
PollSavvy,0.1219,0.0794,-0.3613,-0.2488,46.0
PredictWise Markets,0.1272,0.0767,-0.3929,-0.2523,46.0
PredictWise Overall,0.1276,0.0783,-0.4083,-0.2705,46.0
DeSart and Holbrook,0.1279,0.0825,-0.4053,-0.2708,44.0
Daily Kos,0.1439,0.0864,-0.8772,-0.5629,46.0


In [181]:
latest_2020

,office,state,party,candidate,date,model,win_prob,est_diff,est_share,est_share_2p
0,P,AK,D,Biden,2020-11-03,polls-plus,0.14950,-0.084540,NaN,0.456622
1,P,AK,R,Trump,2020-11-03,polls-plus,0.85050,0.084540,NaN,0.543378
2,P,AL,D,Biden,2020-11-03,polls-plus,0.01645,-0.200006,NaN,0.399223
3,P,AL,R,Trump,2020-11-03,polls-plus,0.98355,0.200006,NaN,0.600777
4,P,AR,D,Biden,2020-11-03,polls-plus,0.01020,-0.226813,NaN,0.384551
...,...,...,...,...,...,...,...,...,...,...
107,P,WI,R,TRUMP,2020-11-03,pec,0.05000,-0.085000,NaN,NaN
108,P,WV,D,BIDEN,2020-11-03,pec,0.05000,-0.201000,NaN,NaN
109,P,WV,R,TRUMP,2020-11-03,pec,0.95000,0.201000,NaN,NaN
110,P,WY,D,BIDEN,2020-11-03,pec,0.00000,-0.305000,NaN,NaN


In [415]:
new_uva_data.drop(new_uva_data[~new_uva_data['state'].isin(ELECTORAL_VOTES.to_dict().keys())].index, inplace = True) 
print(len(new_uva_data))
latest_data["uva"] = new_uva_data
latest_2020 = pd.concat(list(latest_data.values()))
scenario_2020 = Scenario.from_csv(WINNERS_2020_PATH, latest_2020)

102


//anaconda3/envs/py3/lib/python3.7/site-packages/ipykernel_launcher.py:4: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  after removing the cwd from sys.path.


In [416]:
re = scenario_2020.forecast_scores.round(4)\
    .sort_values("pres_brier_evs").fillna("")\
    [[ "pres_brier_evs", "pres_brier_flat", "pres_correct" ]]


In [417]:
re

,pres_brier_evs,pres_brier_flat,pres_correct
pollyvote,0.0575,0.0330,49.0
pec,0.0621,0.0317,49.0
UVA,0.0640,0.0392,49.0
fivethirtyeight-polls-plus,0.0662,0.0359,49.0
economist,0.0709,0.0368,49.0


In [428]:
pollyvote_latest = get_certain(latest_2020, "pollyvote") # .reset_index().drop(columns=["index"])
polls_plus_latest = get_certain(latest_2020, "fivethirtyeight-polls-plus") # .reset_index().drop(columns=["index"])
economist_latest = get_certain(latest_2020, "economist") # .reset_index().drop(columns=["index"])
pec_latest = get_certain(latest_2020, "pec") # .reset_index().drop(columns=["index"])
uva_latest = get_certain(latest_2020, "UVA")

In [491]:
a = uva_latest.sort_values(by='state', ascending=True, na_position='first').reset_index()

In [492]:
a["candidate"] = a["candidate"].str.capitalize()

In [494]:
a.to_csv("UVA.csv")

In [331]:
winners = pd.read_csv(WINNERS_2020_PATH)

In [454]:
def get_prediction_rows(df):
    def selectt(row):
        row["pred"] = row["win_prob"] == max(row["win_prob"])
        return row
    new = df.groupby("state").apply(selectt).reset_index()
#     print(new)
    return new[new["pred"]]

In [456]:
def merge_w_winner(df, winners):
    return pd.merge(df, winners, on=["state"], how="left")

In [457]:
def get_winning_states(df, winners):
    merged_df = merge_w_winner(get_prediction_rows(df), winners)
    return merged_df[merged_df["candidate"] == merged_df["winner"]].state.values

In [458]:
def get_lost_states(df, winners):
    merged_df = merge_w_winner(get_prediction_rows(df), winners)
    return merged_df[merged_df["candidate"].str.upper() != merged_df["winner"].str.upper()].state.values

In [467]:
len(economist_latest)

102

In [473]:
a = get_prediction_rows(polls_plus_latest)
len(a[a["candidate"] == "Biden"])

28

In [471]:
uva_latest

,candidate,date,est_diff,est_share,est_share_2p,model,office,party,prediction,state,win_prob
0,TRUMP,2020-10-27,NaN,NaN,NaN,UVA,P,R,favored,AK,0.8
1,TRUMP,2020-10-27,NaN,NaN,NaN,UVA,P,R,safe,AL,1.0
2,TRUMP,2020-10-27,NaN,NaN,NaN,UVA,P,R,safe,AR,1.0
3,BIDEN,2020-10-27,NaN,NaN,NaN,UVA,P,D,lean,AZ,0.6
4,BIDEN,2020-10-27,NaN,NaN,NaN,UVA,P,D,safe,CA,1.0
5,BIDEN,2020-10-27,NaN,NaN,NaN,UVA,P,D,safe,CO,1.0
6,BIDEN,2020-10-27,NaN,NaN,NaN,UVA,P,D,safe,CT,1.0
7,BIDEN,2020-10-27,NaN,NaN,NaN,UVA,P,D,safe,DC,1.0
8,BIDEN,2020-10-27,NaN,NaN,NaN,UVA,P,D,safe,DE,1.0
9,BIDEN,2020-10-27,NaN,NaN,NaN,UVA,P,D,lean,FL,0.6


In [459]:
print(get_lost_states(pollyvote_latest, winners))
print(get_lost_states(polls_plus_latest, winners))
print(get_lost_states(economist_latest, winners))
print(get_lost_states(pec_latest, winners))
print(get_lost_states(uva_latest, winners))
# scenario_2020 = Scenario.from_csv(WINNERS_2020_PATH, latest_2020)

['FL' 'NC']
['FL' 'NC']
['FL' 'NC']
['FL' 'NC']
['FL' 'NC']


In [343]:
a = scenario_2020.forecast_scores.round(4)\
    .sort_values("pres_brier_evs").fillna("")\
    [[ "pres_brier_evs", "pres_brier_flat", "pres_correct"]].rename(index=TRANSLATIONS)#.to_csv(sys.stdout)

In [348]:
a.rename(columns={"pres_brier_evs": "pres_brier_evs 2020", 
                  "pres_brier_flat": "pres_brier_flat 2020",
                  "pres_correct": "pres_correct 2020"}).drop(index="economist")

,pres_brier_evs 2020,pres_brier_flat 2020,pres_correct 2020
pollyvote,0.0575,0.0330,49.0
Princeton E.C.,0.0621,0.0317,49.0
fivethirtyeight-polls-plus,0.0662,0.0359,49.0


In [386]:
uva_data = pd.read_csv(os.path.join(data_dir, "uva.csv"))

In [387]:
d = {"favored": 0.8, "safe": 1, "lean": 0.6}
d_revert = {0.2: "not favored", 0: "not safe", "0.4": "not lean" }
uva_data_list = uva_data.values.tolist()
lens = len(uva_data_list)
for i in range(lens):
    v = uva_data_list[i]
    s = d[v[-1]]
    v.append(s)
    if v[-3] == "TRUMP":
        uva_data_list.append(v[:4] + ['D', "BIDEN", f"not {v[-2]}", round(1-s, 2)])
    else:
        uva_data_list.append(v[:4] + ['R', "TRUMP", f"not {v[-2]}", round(1-s, 2)])

In [478]:
national_data = [["2020-11-03", "economist", "D", "BIDEN", 0.966],
                ["2020-11-03", "economist", "R", "TRUMP", 0.034],
                ["2020-11-03", "fivethirtyeight-polls-plus", "D", "BIDEN", 0.8916],
                ["2020-11-03", "fivethirtyeight-polls-plus", "R", "TRUMP", 0.1039],
                ["2020-11-03", "pollyvote", "D", "BIDEN", 0.77],
                ["2020-11-03", "pollyvote", "R", "TRUMP", 0.23],
                 ["2020-11-03", "predictit", "D", "BIDEN", 0.42],
                ["2020-11-03", "predictit", "R", "TRUMP", 0.58]]


In [479]:
national_df = pd.DataFrame(national_data, columns=["date", "model", "party", "candidate", "win prob"])

In [480]:
national_df

,date,model,party,candidate,win prob
0,2020-11-03,economist,D,BIDEN,0.9660
1,2020-11-03,economist,R,TRUMP,0.0340
2,2020-11-03,fivethirtyeight-polls-plus,D,BIDEN,0.8916
3,2020-11-03,fivethirtyeight-polls-plus,R,TRUMP,0.1039
4,2020-11-03,pollyvote,D,BIDEN,0.7700
5,2020-11-03,pollyvote,R,TRUMP,0.2300
6,2020-11-03,predictit,D,BIDEN,0.4200
7,2020-11-03,predictit,R,TRUMP,0.5800


In [391]:
new_uva_data = pd.DataFrame(uva_data_list, columns=list(uva_data.columns) + ["win_prob"])

,date,model,office,state,party,candidate,prediction,win_prob
0,2020-10-27,UVA,P,AK,R,TRUMP,favored,0.8
1,2020-10-27,UVA,P,AL,R,TRUMP,safe,1.0
2,2020-10-27,UVA,P,AR,R,TRUMP,safe,1.0
3,2020-10-27,UVA,P,AZ,D,BIDEN,lean,0.6
4,2020-10-27,UVA,P,CA,D,BIDEN,safe,1.0
5,2020-10-27,UVA,P,CO,D,BIDEN,safe,1.0
6,2020-10-27,UVA,P,CT,D,BIDEN,safe,1.0
7,2020-10-27,UVA,P,DC,D,BIDEN,safe,1.0
8,2020-10-27,UVA,P,DE,D,BIDEN,safe,1.0
9,2020-10-27,UVA,P,FL,D,BIDEN,lean,0.6
